In [122]:
%load_ext autoreload
%autoreload 2
import feature_extract
import sys
sys.path.append("..")
import torch
import psql_methods as psql
import pickle
import alchemy_methods as alc
from tqdm import tqdm
import numpy as np
import image_utils as imgs
import Openseas_Methods as opse
import multiprocessing
import pandas as pd
import feature_utils as feat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
f = open('top_100_pfps.txt','r')
slugs = list(f)
slugs = [x.strip('\n') for x in slugs]
commands = ["SELECT slug from nfttoimage group by slug having count(*)<>600"]
data = psql.execute_commands(commands)
data = [x[0] for x in data]
print(len(data))
#Figure out which labels still need to find URLs for
nfts_to_process = [x for x in slugs if x not in data]
nfts_to_process = [x for x in nfts_to_process if x not in opse.SKIP_LIST]
nfts_to_process

9292


['nouns',
 'metahero-generative',
 'jungle-freaks-genesis',
 'bears-deluxe',
 'io-imaginary-ones',
 'mutantcats',
 'kprverse',
 'bossbeauties',
 'theshiboshis']

In [137]:
import logging
logging.basicConfig(filename='counterfeit_url_logs.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
for slug in tqdm(nfts_to_process):
    logging.info(f"Beginning slug {slug}")
    data.append(opse.pull_nft_images(slug,limit_toks=10000))
    logging.info(f"Finished slug {slug}")

 44%|████████████████████████████████████████████▍                                                       | 4/9 [03:29<04:51, 58.37s/it]

integer out of range



 89%|████████████████████████████████████████████████████████████████████████████████████████▉           | 8/9 [08:30<01:12, 72.85s/it]

integer out of range



100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [09:48<00:00, 65.39s/it]


In [148]:
#find out which slugs have already been done
done_slugs = imgs.get_immediate_subdirectories('counterfeit_images/val')
slugs_to_image = [x for x in slugs if x not in done_slugs]
print(slugs_to_image)

['bossbeauties']


In [151]:
#Pull 500 images per NFT collectino 
tup_slugs = tuple(slugs_to_image[0])
command = [f"""WITH numbered_rows AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY slug ORDER BY RANDOM()) AS row_num
  FROM nfttoimage
  where slug in {tup_slugs}
)
SELECT *
FROM numbered_rows
WHERE row_num <= 500;"""]
rows  = psql.execute_commands(command)

In [155]:
rows = [(x[0],x[1],x[2]) for x in rows]
args = imgs.db_to_args(rows)

[]
['slug', 'token_id', 'url']


TypeError: pull_image_from_url() missing 1 required positional argument: 'nft_data'

In [153]:
records = []
num_processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=num_processes) as pool: # Use all cores   
    for result in pool.starmap(imgs.pull_image_from_url, args):
        records.append(result)

TypeError: pull_image_from_url() takes from 2 to 4 positional arguments but 8 were given

In [156]:
imgs.delete_empty_directories('counterfeit_images')

In [163]:
#We can now create a dataframe with th embeddings of all of th top 100 NFTs
#We assume the features have already been computed by the feature_extract_notebook
model_string = 'dinov2_vits14'
data_path = '/global/scratch/tlundy/NFT_Research/nft_research/Dino/counterfeit_images'
out_path = f'/global/scratch/tlundy/NFT_Research/nft_research/Dino/counterfeit_features/{model_string}'
feature_path = out_path+'/testfeat.pth'
features = torch.load(feature_path)
labels = feature_extract.get_labels(data_path)
file_names = feature_extract.get_filenames(data_path)
features_list = features.tolist()
# Create a pandas DataFrame
data = {'Label': labels.tolist(), 'Features': features_list,'Collection':[x[0] for x in file_names],
        'NFT_num':[x[1] for x in file_names]}
df = pd.DataFrame(data)
df


Dataset ReturnIndexDataset
    Number of datapoints: 60382
    Root location: /global/scratch/tlundy/NFT_Research/nft_research/Dino/counterfeit_images/val


,Label,Features,Collection,NFT_num
0,0,"[0.06834631413221359, 0.013368001207709312, 0....",0n1-force,1010
1,0,"[0.08093642443418503, -0.03803253173828125, 0....",0n1-force,1012
2,0,"[0.028013398870825768, -0.02013389579951763, 0...",0n1-force,1025
3,0,"[-0.03390040993690491, 0.004265990108251572, 0...",0n1-force,1034
4,0,"[0.026579292491078377, -0.014232189394533634, ...",0n1-force,1044
...,...,...,...,...
60377,132,"[-0.06007155403494835, 0.01303652860224247, 0....",zombieclub-token,952
60378,132,"[0.008216793648898602, 0.007785656955093145, -...",zombieclub-token,956
60379,132,"[-0.03259636461734772, 0.01624414138495922, 0....",zombieclub-token,964
60380,132,"[0.05409884452819824, -0.014129256829619408, -...",zombieclub-token,989


In [158]:
#Next we load the dataframe containing embeddings of all other collection
df2 = pd.read_pickle('graph_images_dataframe.pkl')
df2

,Label,Features,Collection,NFT_num,AverageFeatureVector
0,0,"[0.006129133980721235, 0.08498869091272354, -0...",-glowa-,1,"[-0.007021066558081657, 0.034550966462120414, ..."
1,0,"[0.04774976521730423, 0.05362828075885773, 0.0...",-glowa-,10017,"[-0.007021066558081657, 0.034550966462120414, ..."
2,0,"[0.10529650747776031, 0.07033471018075943, 0.0...",-glowa-,10018,"[-0.007021066558081657, 0.034550966462120414, ..."
3,0,"[-0.02233046293258667, 0.01190363708883524, -0...",-glowa-,10019,"[-0.007021066558081657, 0.034550966462120414, ..."
4,0,"[0.04859286919236183, 0.054554957896471024, 0....",-glowa-,10024,"[-0.007021066558081657, 0.034550966462120414, ..."
...,...,...,...,...,...
456139,10976,"[-0.033141832798719406, -0.0350814163684845, -...",zzz-zzz-by-hashlips,75,"[-0.033141832798719406, -0.0350814163684845, -..."
456140,10976,"[-0.033141832798719406, -0.0350814163684845, -...",zzz-zzz-by-hashlips,77,"[-0.033141832798719406, -0.0350814163684845, -..."
456141,10976,"[-0.033141832798719406, -0.0350814163684845, -...",zzz-zzz-by-hashlips,78,"[-0.033141832798719406, -0.0350814163684845, -..."
456142,10976,"[-0.033141832798719406, -0.0350814163684845, -...",zzz-zzz-by-hashlips,79,"[-0.033141832798719406, -0.0350814163684845, -..."


In [164]:
df = feat.compute_average_vector(df,column='Label')
df

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 133/133 [00:00<00:00, 178.62it/s]


,Label,Features,Collection,NFT_num,AverageFeatureVector
0,0,"[0.06834631413221359, 0.013368001207709312, 0....",0n1-force,1010,"[0.05036659871786995, -0.013276008738578319, 0..."
1,0,"[0.08093642443418503, -0.03803253173828125, 0....",0n1-force,1012,"[0.05036659871786995, -0.013276008738578319, 0..."
2,0,"[0.028013398870825768, -0.02013389579951763, 0...",0n1-force,1025,"[0.05036659871786995, -0.013276008738578319, 0..."
3,0,"[-0.03390040993690491, 0.004265990108251572, 0...",0n1-force,1034,"[0.05036659871786995, -0.013276008738578319, 0..."
4,0,"[0.026579292491078377, -0.014232189394533634, ...",0n1-force,1044,"[0.05036659871786995, -0.013276008738578319, 0..."
...,...,...,...,...,...
60377,132,"[-0.06007155403494835, 0.01303652860224247, 0....",zombieclub-token,952,"[-0.007146694796058, 0.0027466976506839275, 0...."
60378,132,"[0.008216793648898602, 0.007785656955093145, -...",zombieclub-token,956,"[-0.007146694796058, 0.0027466976506839275, 0...."
60379,132,"[-0.03259636461734772, 0.01624414138495922, 0....",zombieclub-token,964,"[-0.007146694796058, 0.0027466976506839275, 0...."
60380,132,"[0.05409884452819824, -0.014129256829619408, -...",zombieclub-token,989,"[-0.007146694796058, 0.0027466976506839275, 0...."


In [165]:
pw_dists =  feat.pairwise_distances(df,df2,feature_col = 'AverageFeatureVector')
result = pw_dists.groupby('Top_100', group_keys=False).apply(feat.get_smallest_values)
no_dupe = result.query('Top_100!=Alt')
grouped = no_dupe.groupby('Top_100')
root = 'graph_images/val'
for label,group in grouped:
    alt_list = list(group['Alt'])
    slugs = alt_list[:4]+[label]+alt_list[4:]
    print(slugs)
    imgs.create_image_grid(root,slugs,label,'counterfeit_grids',randomize=False)
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [03:17<00:00,  1.47s/it]


['0n1-force', 'neotokyopunkscollabs', 'vainego', 'mekazuki', '.ipynb_checkpoints', 'roars-by-ntp', 'tokyo-brave-heroes', 'skullzuki', 'dreamzukinft', 'tokyobraveheroes2g']
TOO MANY DIRECTORIES NEED TO SET RANDOMIZE TRUE
['neotokyopunkscollabs', 'vainego', 'mekazuki', 'roars-by-ntp', '0n1-force', 'tokyo-brave-heroes', 'skullzuki', 'dreamzukinft', 'tokyobraveheroes2g']
['frnds-club', 'the-pepe-chimpsons', 'toonsquad-official', 'the-chimpsons-official', '3landers', 'we-the-kids', 'colorful-ape-club-official', 'not-a-pepe-nft-project', 'pastel-cronies']
['buds-in-space', 'blitmap', 'dystopunks', 'cryptoskulls', 'acclimatedmooncats', 'pixelchain', 'blitkin', 'n-i-c-k', '-phunks']
['uniqly-collections', 'ethernity-master', 'laliga-meta', '77bit-comics', 'adidasoriginals', 'starman-nft-experience', 'influencerz', 'worldwidewebbitems', 'xociety-frontier']
['heartbreak-bear-gen-2', 'toonsofficialcollection', 'citizenz-nft', 'thealiengirl', 'akidcalledbeast', 'homieg', 'trapmonkie', 'karafuru-x-

In [56]:
no_dupe

,Top_100,Alt,Euc_Distance,Dot_Distance
6569,0n1-force,neotokyopunkscollabs,0.474281,0.887529
10277,0n1-force,vainego,0.477392,0.886049
5810,0n1-force,mekazuki,0.493194,0.878380
8133,0n1-force,roars-by-ntp,0.511942,0.868958
9946,0n1-force,tokyo-brave-heroes,0.513878,0.867964
...,...,...,...,...
1009133,zed-run-official,unifriends-nft-official,0.915407,0.581015
1001629,zed-run-official,di-animals,0.927658,0.569725
1003219,zed-run-official,gunslinger-horses,0.966259,0.533172
1003462,zed-run-official,honeybadgers-art,1.013310,0.486602


In [168]:
pw_dists_no_dupe = pw_dists.query("Top_100!=Alt and Top_100!='.ipynb_checkpoints'")
top_100_slugs = list(pw_dists_no_dupe['Top_100'].unique())
commands = ["SELECT * from collectiontoaddress"]
data = psql.execute_commands(commands)
slug_to_contract = {x[0]:x[1] for x in data}
contracts = []
for slug in top_100_slugs:
    contracts.append((slug,slug_to_contract[slug]))
slug_to_owners = alc.contracts_to_owners(contracts)

 16%|███████████████▎                                                                                 | 21/133 [00:10<00:34,  3.24it/s]

Error on label chromie-squiggle-by-snowfro


 27%|██████████████████████████▎                                                                      | 36/133 [00:17<00:31,  3.09it/s]

Error on label cyberkongz-babies


 33%|████████████████████████████████                                                                 | 44/133 [00:21<00:35,  2.48it/s]

Error on label fidenza-by-tyler-hobbs


 79%|███████████████████████████████████████████████████████████████████████████▊                    | 105/133 [00:50<00:09,  3.02it/s]

Error on label ringers-by-dmitri-cherniak


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 133/133 [01:03<00:00,  2.08it/s]


In [177]:
command = "Select distinct slug from owner_lists"
rows = psql.execute_commands([command])
completed_slugs = [x[0] for x in rows]
completed_slugs

['azuki',
 'meebits',
 'mfers',
 'cyberkongz',
 'collectvoxtownstar',
 'beanzofficial',
 'artgobblers',
 'clonex-mintvial',
 'bored-ape-chemistry-club',
 'adidasoriginals',
 'invisiblefriends',
 'cool-cats-nft',
 'deadfellaz',
 'boredapeyachtclub',
 'doodles-official',
 'cryptopunks',
 'world-of-women-nft',
 'proof-moonbirds',
 'clonex',
 '3landers',
 'alienfrensnft',
 'creatureworld',
 'capsulehouse',
 'milady',
 'mekaverse',
 'azukielementals',
 'cyberkongz-vx',
 'curiocardswrapper',
 'cryptoadz-by-gremplin',
 'hapeprime',
 'goblintownwtf',
 'coolmans-universe',
 '0n1-force',
 'veefriends',
 'bored-ape-kennel-club',
 'thecaptainz',
 'pudgypenguins']

In [ ]:
f = open('top_100_pfps.txt','r')
slugs = list(f)
slugs = [x.strip('\n') for x in slugs]
slugs = [x for x in slugs if x in top_100_slugs]
der_collects = ['mutant-ape-yacht-club','wrapped-cryptopunks','bored-ape-kennel-club','official-v1-punks','moonbirds-oddities','lilpudgys']
for top_slug in tqdm(slugs):
    # Sort the DataFrame by the 'Values' column
    print(top_slug)
    if top_slug in completed_slugs or top_slug in der_collects:
        continue
    own_lists = []
    try:
        wallets = slug_to_owners[top_slug]
    except:
        continue
    print(len(wallets))
    for i,wal in enumerate(wallets):
        if i%100==0:
            print(i)
        own_lists.append(alc.owners_to_NFT([wal]))
    psql_input = []
    own_slugs = [[x[0] for x in sub_list] for sub_list in own_lists if sub_list]
    for wal,own_list in zip(wallets,own_slugs):
        for slug in own_list:
           psql_input.append((slug,wal,top_slug))
    command = "INSERT INTO owner_lists (alt, wallet,slug) VALUES (%s, %s, %s)"
    psql.batch_insert(command,psql_input)

  0%|                                                                                                           | 0/95 [00:00<?, ?it/s]

boredapeyachtclub
cryptopunks
mutant-ape-yacht-club
azuki
clonex
wrapped-cryptopunks
proof-moonbirds
doodles-official
bored-ape-kennel-club
pudgypenguins
meebits
beanzofficial
cool-cats-nft
milady
0n1-force
world-of-women-nft
thecaptainz
mfers
veefriends
goblintownwtf
hapeprime
invisiblefriends
mekaverse
thepotatoz
2644
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600


 25%|████████████████████████                                                                       | 24/95 [47:33<2:20:41, 118.90s/it]

murakami-flowers-seed
330
0
100
200
300


 26%|█████████████████████████                                                                      | 25/95 [52:42<2:31:08, 129.55s/it]

fluf
3230
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200


 27%|█████████████████████████▍                                                                   | 26/95 [2:03:01<8:18:25, 433.41s/it]

pxnghostdivision
4410
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400


 28%|██████████████████████████▏                                                                 | 27/95 [3:46:08<18:36:49, 985.43s/it]

karafuru
3327
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700


In [94]:
psql_input = []
for wal,own_list in zip(wallets,own_slugs):
    for slug in own_list:
       psql_input.append((slug,wal,'milady'))

In [103]:
command = "INSERT INTO owner_lists (alt, wallet,slug) VALUES (%s, %s, %s)"
psql.batch_insert(command,psql_input)

name 'rows' is not defined


In [89]:
top_df = pw_dists_no_dupe.query(f"Top_100=='{top_slug}'")
sorted_df = mil_df.sort_values(by='Euc_Distance')
# Get the sorted labels as a list
sorted_labels = sorted_df['Alt'].tolist()
own_slugs = [[x[0] for x in sub_list] for sub_list in own_lists if sub_list]
seen_labels = []
count_to_overlap = []
for i,label in enumerate(sorted_labels):
    if i%1000==0:
        print(i)
    count = 0
    seen_labels.append(label)
    set1 = set(seen_labels)
    for own_list in own_slugs:
        set2 = set(own_list)
        if len(set1.intersection(set2))>0:
            count+=1
    count_to_overlap.append((i,count))
    if count==len(own_slugs):
        break

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [90]:
top_df = pw_dists_no_dupe.query(f"Top_100=='{top_slug}'")
sorted_df = mil_df.sort_values(by='Euc_Distance')
# Get the sorted labels as a list
sorted_labels = sorted_df['Alt'].tolist()
own_slugs = [[x[0] for x in sub_list] for sub_list in own_lists if sub_list]
seen_labels = []
count_to_overlap = []
for i,label in enumerate(sorted_labels):
    if i%1000==0:
        print(i)
    count = 0
    seen_labels.append(label)
    set1 = set(seen_labels)
    for own_list in own_slugs:
        set2 = set(own_list)
        if len(set1.intersection(set2))>0:
            count+=1
    count_to_overlap.append((i,count))
    if count==len(own_slugs):
        break

[(0, 91),
 (1, 115),
 (2, 149),
 (3, 149),
 (4, 298),
 (5, 340),
 (6, 417),
 (7, 526),
 (8, 545),
 (9, 545),
 (10, 549),
 (11, 552),
 (12, 641),
 (13, 641),
 (14, 648),
 (15, 759),
 (16, 1106),
 (17, 1106),
 (18, 1106),
 (19, 1106),
 (20, 1106),
 (21, 1126),
 (22, 1132),
 (23, 1132),
 (24, 1146),
 (25, 1149),
 (26, 1154),
 (27, 1157),
 (28, 1160),
 (29, 1177),
 (30, 1178),
 (31, 1178),
 (32, 1178),
 (33, 1182),
 (34, 1184),
 (35, 1194),
 (36, 1194),
 (37, 1194),
 (38, 1203),
 (39, 1203),
 (40, 1211),
 (41, 1211),
 (42, 1212),
 (43, 1212),
 (44, 1247),
 (45, 1262),
 (46, 1262),
 (47, 1262),
 (48, 1269),
 (49, 1271),
 (50, 1271),
 (51, 1271),
 (52, 1271),
 (53, 1271),
 (54, 1271),
 (55, 1271),
 (56, 1272),
 (57, 1272),
 (58, 1272),
 (59, 1274),
 (60, 1275),
 (61, 1275),
 (62, 1275),
 (63, 1275),
 (64, 1346),
 (65, 1346),
 (66, 1346),
 (67, 1346),
 (68, 1346),
 (69, 1346),
 (70, 1346),
 (71, 1346),
 (72, 1346),
 (73, 1346),
 (74, 1347),
 (75, 1348),
 (76, 1348),
 (77, 1366),
 (78, 1366),
